In [28]:
### SETUP ###

import arcpy
from arcgis import gis
import pandas as pd
from arcpy.sa import *
import ast

arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'BiodiversityMBRC.gdb'
gdb = arcpy.env.workspace

### FUNCTIONS ###

# spatial join function that allows you to input fields to join
def simpleSpatialJoin(fcA, fcB, fcOut, fieldsToJoin, joinType, keepType):
    # Get field mappings of input dataset
    mapA = arcpy.FieldMappings()
    mapA.addTable(fcA)
    # Get field mappings of join dataset
    mapB = arcpy.FieldMappings()
    mapB.addTable(fcB)
    # Get index of sensitivity score field and add to field map. Define merge rule.
    for field in fieldsToJoin:
        ind = mapB.findFieldMapIndex(field)
        fmap = mapB.getFieldMap(ind)
        fmap.mergeRule = 'First'
        mapA.addFieldMap(fmap)
    # Run spatial join
    if joinType == 'JOIN_ONE_TO_ONE':
        match = 'LARGEST_OVERLAP'
    else:
        match = 'INTERSECT'
    arcpy.analysis.SpatialJoin(fcA, fcB, fcOut, joinType, keepType, mapA, match, None, "")
    delFields = ["Join_Count", "TARGET_FID"]
    for field in delFields:
        arcpy.management.DeleteField(fcOut, field)

# same as above, but with a search radius
def simpleSpatialJoinNear(fcA, fcB, fcOut, fieldsToJoin, joinType, keepType, radius):
    # Get field mappings of input dataset
    mapA = arcpy.FieldMappings()
    mapA.addTable(fcA)
    # Get field mappings of join dataset
    mapB = arcpy.FieldMappings()
    mapB.addTable(fcB)
    # Get index of sensitivity score field and add to field map. Define merge rule.
    for field in fieldsToJoin:
        ind = mapB.findFieldMapIndex(field)
        fmap = mapB.getFieldMap(ind)
        fmap.mergeRule = 'First'
        mapA.addFieldMap(fmap)
    # Run spatial join
    if joinType == 'JOIN_ONE_TO_ONE':
        match = 'CLOSEST'
    else:
        match = 'INTERSECT'
    arcpy.analysis.SpatialJoin(fcA, fcB, fcOut, joinType, keepType, mapA, match, f"{radius} Meters", "")
    
def replaceNone(dataset, field):
    with arcpy.da.UpdateCursor(dataset, field) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0
                cursor.updateRow(row)
                

In [3]:
### VEGETATION - BASE
# Join basemap layer attributes to conservation network map
# !! Large amount of features. Runtime ~100 minutes for 100m grid

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

arcpy.management.MakeFeatureLayer('grid_clip_100m', 'grid')
# core veg
simpleSpatialJoin('grid', 'Basemap/coreVegetationCategorised_clean', 'Processing/gridCore', ['vegetation_cover_type', 'current_re', 'size_category', 'core_area_id'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# stepping stones
simpleSpatialJoin('gridCore', 'Basemap/steppingStonesVegetationCategorised_clean', 'Processing/gridStep', ['size_category', 'stepping_stone_id'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# ecological corridors
arcpy.management.MakeFeatureLayer('Basemap/ecologicalCorridorsCategorised', 'ecoCorridors')
simpleSpatialJoin('gridStep', 'ecoCorridors', 'Processing/gridCorr', ['feature_type', 'eco_corridor_id', 'corridor_width'],'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# remnant REs
simpleSpatialJoin('Processing/gridCorr', 'Input/Biodiversity_status_of_remnant_regional_ecosystems', 'Processing/gridRe', ['RE1', 'RE2', 'RE3', 'RE4', 'RE5', "RE_LABEL"], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
# restoration area
simpleSpatialJoin('Processing/gridRe', 'Basemap/restorationAreas', 'Processing/gridResto', ['restoration_area_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')

# convert to pandas dataframe for column manipulation
grid = pd.DataFrame.spatial.from_featureclass("Processing/gridResto")
print(grid.columns)
grid.rename(columns={'size_category': 'core_size_category',
                    'size_category_1': 'stepping_stone_size_category',
                    'feature_type': 'eco_corridor_feature',
                    'current_re': 'refined_re'},inplace=True)
# specify columns to export
gridSub = grid[['vegetation_cover_type', 'refined_re', 'RE1', 'RE2', 'RE3', 'RE4', 'RE5', 'RE_LABEL', 'core_size_category', 'core_area_id', 'stepping_stone_size_category', 'stepping_stone_id',
                'eco_corridor_feature', 'eco_corridor_id', 'restoration_area_id', 'SHAPE']]
# export to feature class
gridSub.spatial.to_featureclass('BiodiversityMBRC.gdb/Network_grids/vegGrid')
arcpy.management.CalculateField("vegGrid", "grid_id", "!OBJECTID!", "PYTHON", "", "LONG")

Index(['OBJECTID', 'vegetation_cover_type', 'current_re', 'size_category',
       'core_area_id', 'size_category_1', 'stepping_stone_id', 'feature_type',
       'eco_corridor_id', 'corridor_width', 'RE1', 'RE2', 'RE3', 'RE4', 'RE5',
       'RE_LABEL', 'restoration_area_id', 'SHAPE'],
      dtype='object')


<Result 'vegGrid'>

In [5]:
## vegetation grid - add BVG
flds = ['BVG1M', 'BVG1M_PC', 'DBVG1M', 'DBVG2M', 'DBVG5M']
arcpy.management.MakeFeatureLayer('Input/Pre_clearing_Broad_Vegetation_Groups', 'bvg')
arcpy.management.MakeFeatureLayer('Network_grids/vegGrid', 'hvr', "vegetation_cover_type IN ('High value regrowth', 'High value regrowth/Remnant')")
simpleSpatialJoin('hvr', 'bvg', 'Processing/hvrBVG', flds, 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
prefix = 'preclear_bvg'
newfields = []
for field in flds:
    newfield = f'{prefix}_{field}'
    arcpy.management.AlterField('Processing/hvrBVG', field, f'{prefix}_{field}', f'{prefix}_{field}')
    newfields.append(newfield)

arcpy.management.JoinField('Network_grids/vegGrid', 'grid_id', 'Processing/hvrBVG', 'grid_id', newfields)

<Result 'BiodiversityMBRC.gdb\\Network_grids/vegGrid'>

In [24]:
## vegetation grid - move refined_re into the re columns where necessary
    
df = pd.DataFrame.spatial.from_featureclass('Network_grids/vegGrid')

df['re6'] = ''
df['re7'] = ''
df['re8'] = ''
df['re9'] = ''
df['re10'] = ''
df['re11'] = ''

df = df[['OBJECTID', 'vegetation_cover_type', 'refined_re', 're1', 're2', 're3', 're4', 're5', 're6', 're7', 're8', 're9', 're10', 're11',
                                                              're_label', 'core_size_category', 'core_area_id',
                                                              'stepping_stone_size_category', 'stepping_stone_id',
                                                              'eco_corridor_feature', 'eco_corridor_id', 'restoration_area_id', 'grid_id', 'preclear_bvg_bvg1_m', 'preclear_bvg_bvg1_m_pc',
                                                              'preclear_bvg_dbvg1_m', 'preclear_bvg_dbvg2_m', 'preclear_bvg_dbvg5_m', 'SHAPE']]

mask = df['refined_re'].str.contains('/')
df2 = df.loc[mask, 'refined_re'].str.split('/', n=10, expand=True)

df.loc[mask, ['re1', 're2', 're3', 're4', 're5', 're6', 're7', 're8', 're9', 're10', 're11']] = df2.values

print(df.columns)

mask = df['refined_re'] != ''

df.loc[mask, 're_label'] = df.loc[mask, 'refined_re']

df.spatial.to_featureclass(gdb + r'\Network_grids\vegGrid')

with arcpy.da.UpdateCursor('Network_grids/vegGrid', ['refined_re', 're1']) as cursor:
    for row in cursor:
        if not '/' in row[0] and row[0] != '' and row[0] is not None:
            row[0] = row[1]
        cursor.updateRow(row)


Index(['OBJECTID', 'vegetation_cover_type', 'refined_re', 're1', 're2', 're3',
       're4', 're5', 're6', 're7', 're8', 're9', 're10', 're11', 're_label',
       'core_size_category', 'core_area_id', 'stepping_stone_size_category',
       'stepping_stone_id', 'eco_corridor_feature', 'eco_corridor_id',
       'restoration_area_id', 'grid_id', 'preclear_bvg_bvg1_m',
       'preclear_bvg_bvg1_m_pc', 'preclear_bvg_dbvg1_m',
       'preclear_bvg_dbvg2_m', 'preclear_bvg_dbvg5_m', 'SHAPE'],
      dtype='object')


In [25]:
### FAUNA HABITAT AND MOVEMENT

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# !! will have to join grid to points. Then join points to grid.

# save FMI datasets to list
fmi = arcpy.ListFeatureClasses('FMI*', '', 'Input')
for thisFMI in fmi:
    if thisFMI == fmi[0]:
        simpleSpatialJoinNear("Network_grids/vegGrid", f'Input/{thisFMI}', f'Processing/grid_join_{thisFMI}', ['SITE', 'ROAD', 'LOCATION', 'E', 'N'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
        lastFMI = thisFMI
    else:
        simpleSpatialJoinNear(f"Processing/grid_join_{lastFMI}", f'Input/{thisFMI}', f'Processing/grid_join_{thisFMI}', ['SITE', 'ROAD', 'LOCATION', 'E', 'N'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
        lastFMI = thisFMI
    df = pd.DataFrame.spatial.from_featureclass(f'Processing/grid_join_{thisFMI}').drop(columns=['Join_Count', 'TARGET_FID'])
    df.rename(columns={'site': f'{thisFMI}_site',
                      'road': f'{thisFMI}_road',
                      'location': f'{thisFMI}_location',
                       'e': f'{thisFMI}_e',
                       'n': f'{thisFMI}_n'},inplace=True)
    df.spatial.to_featureclass(f"BiodiversityMBRC.gdb/Processing/grid_join_{thisFMI}")

In [26]:
# fauna habitat and movement - FMI point datasets
if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

fmi = arcpy.ListFeatureClasses('FMI*', '', 'Input')

arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/fmiGrid")
grid = pd.DataFrame.spatial.from_featureclass("fmiGrid")

for thisFMI in fmi:
    output = f"Processing/{thisFMI}_join_grid"
    simpleSpatialJoinNear(f"Input/{thisFMI}", "Processing/fmiGrid", output, ['grid_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
    df = pd.DataFrame.spatial.from_featureclass(output)
    # drop null grid ids
    df = df.dropna(subset = 'grid_id')
    # convert columns to strings
    df = df.astype(str)
    # concatenate e and n columns
    df['e_n'] = '(' + df['e'].astype(str) + ', ' + df['n'].astype(str) + ')'
    # group by grid id and summarise relevant attributes
    df = df.groupby(by = 'grid_id', as_index=False).agg(lambda x: '; '.join(set(x))).reset_index()
    # convert grid id back to int
    df['grid_id'] = df['grid_id'].astype(int)
    # join columns to grid dataset
    grid = pd.merge(grid, df[['site', 'road', 'location', 'e_n', 'grid_id']], on='grid_id', how='left')
    grid.rename(columns={'site': f'{thisFMI}_site',
                      'road': f'{thisFMI}_road',
                      'location': f'{thisFMI}_location',
                       'e_n': f'{thisFMI}_e_n'},inplace=True)

grid.spatial.to_featureclass(f"BiodiversityMBRC.gdb/Processing/fmiGrid")

'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Processing\\fmiGrid'

In [28]:
# fauna habitat and movement - spatial join habitats - MUST RUN PREVIOUS CHUNK

arcpy.management.MakeFeatureLayer("Processing/fmiGrid", "fmiGrid")

# merge koala habitat
try:
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')
except:
    arcpy.management.Delete("Processing/Koala_habitat")
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')
    
# spatial join koala
try:
    simpleSpatialJoin("fmiGrid", "Koala_habitat", "Processing/fmiKoala", ["Dataset", "KPA", "HSM_CATEGORY"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
except:
    arcpy.management.Delete("Processing/fmiKoala")
    simpleSpatialJoin("fmiGrid", "Koala_habitat", "Processing/fmiKoala", ["Dataset", "KPA", "HSM_CATEGORY"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"Dataset": "koala_dataset",
              "KPA": "koala_priority_area",
              "HSM_CATEGORY": "koala_habitat_suitability"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("fmiKoala", oldField, newField, newField)
    
# spatial join platypus
arcpy.management.MakeFeatureLayer("Input/Platypus_Habitat", "Platypus_Habitat")
try:
    simpleSpatialJoin("fmiKoala", "Platypus_Habitat", "Processing/fmiPlat", ["habitat"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
except:
    arcpy.management.Delete("Processing/fmiPlat")
    simpleSpatialJoin("fmiKoala", "Platypus_Habitat", "Processing/fmiPlat", ["habitat"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"habitat": "platypus_habitat"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("fmiPlat", oldField, newField, newField)

# spatial join turtle nesting areas
arcpy.management.MakeFeatureLayer("Input/Sea_turtle_nesting_areas", "turtles")
simpleSpatialJoin("fmiPlat", "turtles", "Processing/fmiTurtle", ["SIGNIFICANCE"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
    
# rename join fields
alterFields = {"SIGNIFICANCE": "sea_turtle_habitat_significance"}

for oldField, newField in alterFields.items():
    arcpy.management.AlterField("fmiTurtle", oldField, newField, newField) 
    
# add shorebird roosts
arcpy.management.SelectLayerByLocation("fmiTurtle", "INTERSECT", "Input/Shorebird_Roosts", None, "NEW_SELECTION")
arcpy.management.CalculateField("fmiTurtle", "shorebird_roosts", "'Present'", "PYTHON3")
arcpy.management.SelectLayerByAttribute("fmiTurtle", "CLEAR_SELECTION")

# add shorebird known habitat
arcpy.management.MakeFeatureLayer('Input/Shorebird_Known_Habitat_stats_01_2024', 'shorebird_known')
simpleSpatialJoin('fmiTurtle', 'shorebird_known', 'Processing/fmiKnown', ['Site_Name', 'Hab_Type'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
renameDict = {
    'Site_Name': 'hws_sbird_hab_kwn_site',
    'Hab_Type': 'hws_sbird_hab_kwn_type'
}
for old, new in renameDict.items():
    arcpy.management.AlterField('fmiKnown', old, new, new)
    
# add shorebird potential habitat
arcpy.management.MakeFeatureLayer('Input/Shorebird_Potential_Habitats_01_2024', 'shorebird_pot')
simpleSpatialJoin('fmiKnown', 'shorebird_pot', 'Processing/fmiPot', ['Hab_Type'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
renameDict = {
    'Hab_Type': 'hws_sbird_hab_potential_type'
}
for old, new in renameDict.items():
    arcpy.management.AlterField('fmiPot', old, new, new)

# copy output grid
try:
    arcpy.management.CopyFeatures('fmiPot', 'Netword_grids/faunaMovementGrid')
except:
    arcpy.management.Delete('Network_grids/faunaMovementGrid')
    arcpy.management.CopyFeatures('fmiPot', 'Network_grids/faunaMovementGrid')

ExecuteError: ERROR 000210: Cannot create output Netword_grids/faunaMovementGrid
Failed to execute (CopyFeatures).


In [ ]:
### LAND USE
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'BiodiversityMBRC.gdb'
gdb = arcpy.env.workspace

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create land use grid, set extent
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/luGrid")
arcpy.env.extent = "Processing/luGrid"

# create merged council owned land dataset
if not arcpy.Exists('Input/councilOwnedLand'):
    arcpy.management.SelectLayerByLocation('Input/Registered_Parcels_Council_Owned_Managed_OwnerJoin', 'INTERSECT', 'ttePoint')
    arcpy.management.Merge('Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Civic;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Rec_and_Foreshore;Input/LGIP_PPCF_Existing_Trunk_Public_Parks_Sports', 
                           'Processing/LGIP_merged',  None, 'NO_SOURCE_INFO')

    # pull out council land from MBRC Parcels
    arcpy.management.MakeFeatureLayer('Input/MBRCParcels_Registered_Parcels', 'councilParcels', "COUNCIL_LAND IN ('Council Freehold', 'Council reserve', 'Council Leasee')")
    arcpy.analysis.PairwiseErase('councilParcels', 'Processing/LGIP_merged', 'Processing/councilParcels_Erase')
    arcpy.management.Merge('Processing/councilParcels_Erase;Processing/LGIP_merged', 'Processing/councilOwnedLand_WIP', None, 'NO_SOURCE_INFO')
    
    # pull out TTE parcels that don't overlap with any other parcels in merged dataset
    arcpy.management.FeatureToPoint('Input/Registered_Parcels_Council_Owned_Managed_OwnerJoin', 'Processing/ttePoint', "INSIDE")
    arcpy.management.SelectLayerByLocation('ttePoint', 'INTERSECT', 'councilOwnedLand_WIP', '', 'NEW_SELECTION', 'INVERT')
    arcpy.management.SelectLayerByLocation('Input/Registered_Parcels_Council_Owned_Managed_OwnerJoin', 'INTERSECT', 'ttePoint')
    arcpy.management.Merge('Input/Registered_Parcels_Council_Owned_Managed_OwnerJoin;councilOwnedLand_WIP', 'Input/councilOwnedLand', None, 'NO_SOURCE_INFO')

# join council owned land and state-owned land to grid
jDict = {
    'councilOwnedLand':{
        'colname': 'council_land',
        'fields': ['COUNCIL_LAND', 'LANDUSE', 'TENURE']
    },
    'StateLand_Cadastral_data_QLD_CADASTRE_DCDB': {
        'colname': 'state_land',
        'fields': ['TENURE']
        }
}

i = 0
for dset, attr in jDict.items():
    thisLayer = attr['colname']
    if i == 0:
        grid = 'luGrid'
    else:
        grid = output
    output = f'Processing/{dset}_grid'
    simpleSpatialJoin(grid, f'Input/{dset}', output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1
        
# join green infrastructure eco connections to grid
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_Ecological_connections", "green_eco")
simpleSpatialJoin(output, "green_eco", "Processing/green_eco_grid", ["Name", "Type", "Class"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
# rename fields
rename = {"Name": "g_i_eco_connection_name",
         "Type": "g_i_eco_connection_type",
         "Class": "g_i_eco_connection_class"}
for old, new in rename.items():
    arcpy.management.AlterField("green_eco_grid", old, new, new)

# join green infrastructure waterways/drainage to grid
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_Waterways_and_drainage_lines", "green_ww")
simpleSpatialJoin("green_eco_grid", "green_ww", "Processing/green_ww_grid", ["OVL2_CAT", "Name", "CATCHMENT", "STREAM_ID"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
# rename fields
rename = {"OVL2_CAT": "g_i_waterway_ovl2_cat",
         "Name": "g_i_waterway_name",
         "CATCHMENT": "g_i_waterway_catchment",
         "STREAM_ID": "g_i_waterway_stream_id"}
for old, new in rename.items():
    arcpy.management.AlterField("green_ww_grid", old, new, new)
    
# join green infrastructure managed forest to grid. This dataset has multiple overlapping polygons, therefore we will create a grid column presence/absence for each of the four types:
#  -Protected area
# - Managed forest
# - Green infrastructure
# - Reserves of conservation significance
arcpy.management.MakeFeatureLayer("Input/GreenInfrastructure_GI_and_Managed_Forest", "forest")
sets = {'Green Infrastructure': 'g_i_area',
       'Managed Forest': 'g_i_managed_forest',
       'Protected Areas': 'g_i_protected_areas',
       'Reserves of Conservation Significance': 'g_i_sig_conservation_reserves'}

for subcat, colname in sets.items():
    arcpy.management.SelectLayerByAttribute("forest", "NEW_SELECTION", f"SubCategory = '{subcat}'")
    arcpy.management.SelectLayerByLocation("green_ww_grid", "INTERSECT", "forest", "", "NEW_SELECTION")
    arcpy.management.CalculateField("green_ww_grid", colname, "'Present'", "PYTHON3")

arcpy.management.SelectLayerByAttribute("green_ww_grid", "CLEAR_SELECTION")

# join local precincts to grid
arcpy.management.MakeFeatureLayer("Input/PlanningScheme_Local_Plan_Precincts", "precincts")
simpleSpatialJoin("green_ww_grid", "precincts", "Processing/precincts_grid", ["LP", "LP_PREC"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
rename = {"LP": "local_plan",
         "LP_PREC": "local_plan_prec"}
for old, new in rename.items():
    arcpy.management.AlterField("precincts_grid", old, new, new)

# join planning scheme datasets to grid
planDict = {
    'PlanningScheme_Zones': {
        'colname': 'plan_scheme_zones',
        'fields': ['LP', 'LP_PREC', 'ZONE_PREC', 'LVL1_ZONE']
    },
    'PlanningScheme_Priority_Development_Area': {
        'colname': 'plan_scheme_dev_area',
        'fields': ['PDA']
    },
    'PlanningSchemeEAs_Caboolture_West_Local_Plan': {
        'colname': 'plan_scheme_local_plan',
        'fields': ['LP']
    },
    'PlanningSchemeEAs_Waterways': {
        'colname': 'plan_scheme_ww',
        'fields': ['CAT_DESC', 'OVL2_DESC', 'Name', 'CATCHMENT', 'Env_Buffer_m']
    },
    'PlanningScheme_North_Lakes_Locality': {
        'colname': 'plan_scheme_north_lakes',
        'fields': ['Mask']
    },
    'PlanningScheme_Strategic_Framework_Place_Types': {
        'colname': 'plan_scheme_strat_fw',
        'fields': ['PLACETYPE']
    },
    'MBRC_Planning_Scheme___Riparian_Wetland_Setbacks_Overlay_Waterways': {
        'colname': 'plan_scheme_r_w',
        'fields': ['Name', 'Dev_SetBac', 'Env_Buffer', 'STREAM_ID', 'STREAM_NO']
    },
    'MBRC_Planning_Scheme___Riparian_Wetland_Setbacks_Overlay_areas': {
        'colname': 'plan_scheme_setback',
        'fields': ['OVL2_CAT']
    }
}

i = 0
# join planning scheme datasets to grid
for dset, attr in planDict.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f'Input/{dset}', thisLayer)
    if i == 0:
        thisGrid = 'precincts_grid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1

# Join planning scheme EA datasets to grid
arcpy.management.MakeFeatureLayer('Input/PlanningSchemeEAs_Environmental_areas', 'ea_mles', "OVL2_DESC LIKE 'MLES%'")
simpleSpatialJoin(output, "ea_mles", "Processing/ea_mles_grid", ['OVL2_DESC'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
arcpy.management.AlterField("ea_mles_grid", "OVL2_DESC", 'plan_scheme_ea_mles_ovl2_desc', 'plan_scheme_ea_mles_ovl2_desc')
arcpy.AddMessage([field.name for field in arcpy.ListFields('ea_mles')])


arcpy.management.MakeFeatureLayer('Input/PlanningSchemeEAs_Environmental_areas', 'ea_mses', "OVL2_DESC LIKE 'MSES%'")
simpleSpatialJoin('ea_mles_grid', "ea_mses", "Processing/ea_mses_grid", ['OVL2_DESC'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
arcpy.management.AlterField("ea_mses_grid", "OVL2_DESC", 'plan_scheme_ea_mses_ovl2_desc', 'plan_scheme_ea_mses_ovl2_desc')
arcpy.AddMessage([field.name for field in arcpy.ListFields('ea_mses')])


# join regional land use category to grid. There are overlapping polygons, so pull out each land use individually
rluc = {'Regional Landscape and Rural Production Area': 'rluc_landscape_rpa',
 'Rural Living Area':'rluc_rural_living',
 'Urban Footprint':'rluc_urban_footprint'}
for key, value in rluc.items():
    arcpy.management.MakeFeatureLayer("Input/Regional_land_use_categories___South_East_Queensland_Regional_Plan_2023___ShapingSEQ", value, f"RLUC2023 = '{key}'")
    arcpy.management.SelectLayerByLocation('ea_mses_grid', 'INTERSECT', value)
    arcpy.management.CalculateField("ea_mses_grid", f"{value}", "'Yes'", 'PYTHON3')
    arcpy.management.SelectLayerByAttribute('ea_mses_grid', 'CLEAR_SELECTION')

# join development areas to grid
arcpy.management.MakeFeatureLayer("Input/Development_area___South_East_Queensland_Regional_Plan_2023___ShapingSEQ", "dev")
simpleSpatialJoin("ea_mses_grid", "dev", "Processing/dev_grid", ["NAME"], "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.AlterField("dev_grid", "NAME", "shaping_seq_develop_area", "shaping_seq_develop_area")

# add MSES datasets to grid
# create dictionary to run for loop to add all MSES datasets to grid. Includes original dataset name, update column prefix and fields to join.
mses = {
    'MSES___High_ecological_value_waters___watercourse': {
        'colname': 'mses_hev_watercourse',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_value_waters___wetlands': {
        'colname': 'mses_hev_wetlands',
        'fields': ['MI_TYPE']
    },
    'MSES___High_ecological_significance_wetlands': {
        'colname': 'mses_hes_wetlands',
        'fields': ['SIGNIFICANCE']
    },
    'MSES___Regulated_vegetation___essential_habitat': {
        'colname': 'mses_rv_ehab',
        'fields': ['EHAB', 'REGROWTH', 'RVM_CAT']
    },
    'MSES___Protected_area___nature_refuges': {
        'colname': 'mses_pa_nat_refuges',
        'fields': ['NAME', 'PLAN_NUM', 'PA_NUM', 'ALT_NAME']
    },
    'MSES___Regulated_vegetation___category_C_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_c_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___Regulated_vegetation___category_B_endangered_or_of_concern': {
        'colname': 'mses_rv_cat_b_eoc',
        'fields': ['RE', 'PC_LABEL', 'VM_CLASS', 'VERSION', 'SOURCE', 'VM_STATUS']
    },
    'MSES___wildlife_habitat___special_least_concern_animal': {
        'colname': 'mses_hab_slc',
        'fields': ['MSES_SLC_SBH', 'MSES_SLC_REC']
    },
    'MSES___wildlife_habitat___endangered_or_vulnerable_wildlife': {
        'colname': 'mses_hab_cev',
        'fields': ['MSES_CEV_HSM', 'MSES_CEV_SBH', 'MSES_CEV_REC']
    },
    'MSES___Legally_secured_offset_area___vegetation_offsets': {
        'colname': 'mses_veg_offset_area',
        'fields': ['PMAV_NO']
    }
}

i = 0
# run for loop to join all MSES datasets to grid
for dset, attr in mses.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'dev_grid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        if field == 'PC_LABEL':
            arcpy.management.AlterField(output, field, f'{thisLayer}_PERCENT', f'{thisLayer}_PERCENT')
        else:
            arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1
    
# add threatened ecological communities to grid
arcpy.management.JoinField(output, 're1', 'MNES_threatened_ecological_communities', 'Regional_ecosystem', ['MNES_community', 'Status', 'Comments__limits'])
rename = {'MNES_community': 'mnes_eco_community',
         'Status': 'mnes_eco_status',
         'Comments__limits': 'mnes_eco_comments'}
for old, new in rename.items():
    arcpy.AlterField(output, old, new, new)

# export land use grid
try:
    arcpy.management.CopyFeatures(output, "Network_grids/landUseGrid")
except:
    arcpy.management.Delete("Network_grids/landUseGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/landUseGrid")
    
print('Land use grid created successfully')

In [6]:
## GDE

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)


# create gde grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/gdeWIPGrid")

# get gde feature classes
gdes = {
    'QLD_GDE_SURFACE_POINTS': 'gde_point',
    'QLD_GDE_SURFACE_LINES': 'gde_line',
    'QLD_GDE_POTENTIAL_AQUIFER': 'gde_pot_aquifer',
    'QLD_GDE_TERRESTRIAL': 'gde_terrestrial',
    'QLD_GDE_SURFACE_AREAS': 'gde_surface_area'
}

i = 0
# add gde layers to grid
for dset, layer in gdes.items():
    arcpy.management.MakeFeatureLayer(f'Input/{dset}', layer)
    if dset == 'QLD_GDE_SURFACE_POINTS':
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "RULE_PART", "VENT_ID", "NAME", "ELEVATION", "LATITUDE", "LONGITUDE"]
    elif dset == 'QLD_GDE_POTENTIAL_AQUIFER':
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "AQ_NAME", "GW_SALINTY", "GW_PH", "GW_RECHARG"]
    else:
        fields = ["GDE_CLASS", "RULE_ID", "RULE_NAME", "RULE_PART"]
    if i == 0:
        thisGrid = "gdeWIPGrid"
    else:
        thisGrid = output
    output = f"Processing/{layer}_grid"
    simpleSpatialJoin(thisGrid, layer, output, fields, "JOIN_ONE_TO_ONE", "KEEP_ALL")
    for field in fields:
        arcpy.management.AlterField(output, field, f"{layer}_{field}", f"{layer}_{field}")
    i += 1

try:
    arcpy.management.CopyFeatures(output, "Network_grids/gdeGrid")
except:
    arcpy.management.Delete("Network_grids/gdeGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/gdeGrid")

In [8]:
## COASTAL AND HYDRO FEATURES

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP coastal hydro grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/hydroWIPGrid")

arcpy.env.extent = "hydroWIPGrid"

# add coastal landform and ripiarian landform attributes
arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "NEW_SELECTION", "re_label LIKE '%12.1.5%'")
arcpy.management.CalculateField("hydroWIPGrid", "coastal_landform", "'Yes'", 'PYTHON3')

arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "NEW_SELECTION", "re_label LIKE '%12.3.5%'")
arcpy.management.CalculateField("hydroWIPGrid", "riparian_floodplain_landform", "'Yes'", 'PYTHON3')

# add coastal management districts to grid
arcpy.management.MakeFeatureLayer('Input/Coastal_plan_coastal_management_district', 'cmd')
arcpy.management.SelectLayerByLocation("hydroWIPGrid", "INTERSECT", "cmd", None, "NEW_SELECTION")
arcpy.management.CalculateField("hydroWIPGrid", "coastal_management_district", "'Yes'", 'PYTHON3')
arcpy.management.SelectLayerByAttribute("hydroWIPGrid", "CLEAR_SELECTION")

# create barrier works category
arcpy.management.MakeFeatureLayer("Input/Queensland_waterways_for_waterway_barrier_works", "barrier")
arcpy.management.CalculateField("barrier", "category", "''", "PYTHON3")
with arcpy.da.UpdateCursor("barrier", ["FISH_PASS", "category"]) as cursor:
    for row in cursor:
        if row[0] == 5:
            row[1] = "Major (Tidal)"
        elif row[0] == 4:
            row[1] = "Major"
        elif row[0] == 3:
            row[1] = "High"
        elif row[0] == 2:
            row[1] = "Moderate"
        elif row[0] == 1:
            row[1] = "Low"
        else:
            row[1] = "N/A"
        cursor.updateRow(row)

# create dictionary layers/fields map dictionary
hydro = {
    'Watercourse_identification_map___downstream_limits': {
        'colname': 'downstream_limit',
        'fields': ['NAME', 'OFFICE', 'REFERENCE', 'DETERMINED', 'CERTIFIED', 'URL'] 
    },
    'Water_storage_points': {
        'colname': 'w_storage_point',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterpoints': {
        'colname': 'w_point',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterholes': {
        'colname': 'waterhole',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Waterfalls': {
        'colname': 'waterfall',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Queensland_waterways_for_waterway_barrier_works': {
        'colname': 'ww_barrier_works',
        'fields': ['category']
    },
    'Major_watercourse_lines': {
        'colname': 'major_watercourse',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Watercourse_lines_NE_Coast_drainage_division___southern_section': {
        'colname': 'watercourse',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Canal_lines': {
        'colname': 'canal_line',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Pondage': {
        'colname': 'pondage',
        'fields': ['FEATURETYPE']
    },
    'Reservoirs': {
        'colname': 'reservoir',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Watercourse_areas': {
        'colname': 'watercourse_area',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Lakes': {
        'colname': 'lake',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Flats': {
        'colname': 'flat',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'Canal_areas': {
        'colname': 'canal_area',
        'fields': ['FEATURETYPE', 'NAME']
    },
    'MBRC_PlanningScheme_OM_Stormwater_Catchments': {
        'colname': 'stormw_catchment',
        'fields': ['OVL2_DESC']
    }
}

# join attributes to grid using dictionary
i = 0
for dset, attr in hydro.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = 'hydroWIPGrid'
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1

# copy output grid to grid FD
try:
    arcpy.management.CopyFeatures(output, "Network_grids/coastalHydroGrid")
except:
    arcpy.management.Delete("Network_grids/coastalHydroGrid")
    arcpy.management.CopyFeatures(output, "Network_grids/coastalHydroGrid")


In [31]:
## SPECIES OBSERVATION

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP species grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/speciesWIPGrid")

grid = pd.DataFrame.spatial.from_featureclass("speciesWIPGrid")

# merge species observation layers
if not arcpy.Exists('Input/alaMerged'):
    arcpy.management.Merge("Input/ALA_records_20240103;Input/ALA_records_20240206", "Input/alaMerged", "", "ADD_SOURCE_INFO")

dsets = ['alaMerged', 'Wildnet_equalto_prioritysp']

for dset in dsets:
    output = f"Processing/{dset}_join_grid"
    simpleSpatialJoinNear(f"Input/{dset}", "Processing/speciesWIPGrid", output, ['grid_id'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL', 20)
    df = pd.DataFrame.spatial.from_featureclass(output)
    # drop null grid ids
    df = df.dropna(subset = 'grid_id')
    # convert columns to strings
    df = df.astype(str)
    # group by grid id and summarise relevant attributes
    df = df.groupby(by = 'grid_id', as_index=False).agg(lambda x: '; '.join(set(x))).reset_index()
    # convert grid id back to int
    df['grid_id'] = df['grid_id'].astype(int)
    # join columns to grid dataset
    if dset == dsets[0]:
        cols = ['grid_id', 'vernacularName','scientificName']
    else:
        cols = ['grid_id', 'SCI_NAME', 'COM_NAME']
    grid = pd.merge(grid, df[cols], on='grid_id', how='left')
    
grid.rename(columns={'SCI_NAME': 'wildnet_obs_sci_name',
                     'COM_NAME': 'wildnet_obs_com_name',
                     'vernacularName': 'ala_obs_com_name',
                     'scientificName': 'ala_obs_sci_name'},inplace=True)

grid.spatial.to_featureclass('BiodiversityMBRC.gdb/Network_grids/speciesObservationGrid')



[31]:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Network_grids\\speciesObservationGrid'

In [34]:
## SPECIES LIKELIHOOD

##  build species and re dictionaries
# get RE-species matrix
evnt = "evntSpeciesTransposed_cleaned"
cols = [field.name for field in arcpy.ListFields(evnt)]
cols.remove('re')
cols.remove('OBJECTID')
print(cols)
res = []

# List of species names: Common name (scientific name)
species = [
    'Swamp Crayfish (Tenuibranchiurus glypticus)',
    'Richmond Birdwing Butterfly (Ornithoptera richmondia)',
    'Wallum Rocketfrog (Litoria freycineti)',
    'Wallum Sedgefrog (Litoria olongburensis)',
    'Cascade Treefrog (Litoria pearsoniana)',
    'Tusked Frog (Adelotus brevis)',
    'Wallum Froglet (Crinia tinnula)',
    'Common Death Adder (Acanthophis antarcticus)',
    'Eastern Osprey (Pandion cristatus)',
    'Fork-tailed Swift (Apus pacificus)',
    'Spine-tailed Swift (Hirundapus caudacutus)',
    'Beach Stone-curlew (Esacus magnirostris)',
    'Glossy Black-cockatoo (Calyptorhynchus lathami)',
    'Banded Dotterel (Charadrius bicinctus)',
    'Greater Sand Plover (Charadrius leschenaultii)',
    'Lesser Sand Plover (Charadrius mongolus)',
    'Pacific Golden Plover (Pluvialis fulva)',
    'Oriental Cuckoo (Cuculus optatus)',
    'White-winged Black Tern (Chlidonias leucopterus)',
    'Gull-billed Tern (Gelochelidon nilotica)',
    'Caspian Tern (Hydroprogne caspia)',
    'Common Tern (Sterna hirundo)',
    'Little Tern (Sternula albifrons)',
    'Crested Tern (Thalasseus bergii)',
    'Black-faced Monarch (Monarcha melanopsis)',
    'Satin Flycatcher (Myiagra cyanoleuca)',
    'Spectacled Monarch (Symposiachrus trivirgatus)',
    'Eastern Yellow Wagtail (Motacilla flava)',
    'Plumed Frogmouth (Podargus ocellatus plumiferus)',
    'Sooty Shearwater (Ardenna tenuirostris)',
    'Swift Parrot (Lathamus discolor)',
    'Rufous Fantail (Rhipidura rufifrons)',
    'Australian Painted Snipe (Rostratula australis)',
    'Ruddy Turnstone (Arenaria interpres)',
    'Sharp-tailed Sandpiper (Calidris acuminata)',
    'Red Knot (Calidris canutus)',
    'Curlew Sandpiper (Calidris ferruginea)',
    'Red-necked Stint (Calidris ruficollis)',
    'Great Knot (Calidris tenuirostris)',
    "Latham's Snipe (Gallinago hardwickii)",
    "Swinhoe's Snipe (Gallinago megala)",
    'Asian Dowitcher (Limnodromus semipalmatus)',
    'Western Alaskan Bar-tailed Godwit (Limosa lapponica baueri)',
    'Black-tailed Godwit (Limosa limosa)',
    'Eastern Curlew (Numenius madagascariensis)',
    'Little Whimbrel (Numenius minutus)',
    'Whimbrel (Numenius phaeopus)',
    'Ruff (Philomachus pugnax)',
    'Siberian (Grey-tailed) Tattler (Tringa brevipes)',
    'Wood Sandpiper (Tringa glareola)',
    'Common Greenshank (Tringa nebularia)',
    'Marsh Sandpiper (Tringa stagnatilis)',
    'Terek Sandpiper (Xenus cinereus)',
    'Powerful Owl (Ninox strenua)',
    'Glossy Ibis (Plegadis falcinellus)',
    'Black-breasted Button-quail (Turnix melanogaster)',
    'Spotted-tailed Quoll (Dasyurus maculatus maculatus)',
    'Water Mouse (Xeromys myoides)',
    'Platypus (Ornithorhynchus anatinus)',
    'Koala (Phascolarctos cinereus)',
    'Long-nosed Potoroo (Potorous tridactylus)',
    'Southern Greater Glider (Petauroides volans volans)',
    'Grey-headed Flying-fox (Pteropus poliocephalus)',
    'Short-beaked Echidna (Tachyglossus aculeatus)'
]

species_values = [
    'E', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'SL', 'SL', 'SL', 'V', 'V', 'SL', 'V', 'E', 'SL', 'SL', 'SL', 'SL',
    'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'V', 'SL', 'E', 'SL', 'V', 'SL', 'SL', 'E', 'E', 'SL', 'E', 'SL',
    'SL', 'SL', 'V', 'SL', 'E', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'V', 'SL', 'V', 'V', 'V', 'SL', 'V',
    'V', 'V', 'SL'
]

# create dictionary with species and corresponding species conservation status values (NC ACT)
sDict = dict(zip(species, species_values))

# map column names in matrix to full names
fDict = {}
i = 0
for col in cols:
    fDict[col] = species[i]
    i += 1

# build list of unique REs
with arcpy.da.SearchCursor(evnt, "re") as cursor:
    for row in cursor:
        if row[0] not in res:
            res.append(row[0])

# build dictionary, where each key is an RE, and the values are a list of the corresponding species
reDict = {key: [] for key in res}

for col in cols:
    with arcpy.da.SearchCursor(evnt, ["re", col]) as cursor:
        for row in cursor:
            if row[1] == 1:
                reDict[row[0]].append(fDict[col])
                
print(sDict)
print(fDict)
print(reDict)
                

['swampCrayfish', 'richmondBirdwingButterfly', 'wallumRocketfrog', 'wallumSedgefrog', 'cascadeTreefrog', 'tuskedFrog', 'wallumFroglet', 'commonDeathAdder', 'easternOsprey', 'forkTailedSwift', 'spineTailedSwift', 'beachStoneCurlew', 'glossyBlackCockatoo', 'bandedDotterel', 'greaterSandPlover', 'lesserSandPlover', 'pacificGoldenPlover', 'orientalCuckoo', 'whiteWingedBlackTern', 'gullBilledTern', 'caspianTern', 'commonTern', 'littleTern', 'crestedTern', 'blackFacedMonarch', 'satinFlycatcher', 'spectacledMonarch', 'easternYellowWagtail', 'plumedFrogmouth', 'sootyShearwater', 'swiftParrot', 'rufousFantail', 'australianPaintedSnipe', 'ruddyTurnstone', 'sharpTailedSandpiper', 'redKnot', 'curlewSandpiper', 'redNeckedStint', 'greatKnot', 'lathamsSnipe', 'swinhoesSnipe', 'asianDowitcher', 'westernAlaskanBarTailedGodwit', 'blackTailedGodwit', 'easternCurlew', 'littleWhimbrel', 'whimbrel', 'ruff', 'siberianGreyTailedTattler', 'woodSandpiper', 'commonGreenshank', 'marshSandpiper', 'terekSandpiper',

In [36]:
### species likelihood grid
# !! MUST RUN PREVIOUS CHUNK TO BUILD DICTIONARIES

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP species grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/speciesWIPGrid")
grid = 'speciesWIPGrid'

# create separate columns for each species in NC Act status
newcols = ['species_e', "species_v", 'species_sl']
for col in newcols:
    if col == 'species_e':
        status = 'E'
    elif col == 'species_v':
        status = 'V'
    elif col == 'species_sl':
        status = 'SL'
    # filter species dictionary for the target status of this loop iteration
    subSpecies  = [key for key in sDict.keys() if sDict[key] == status]
    arcpy.management.AddField(grid, col, 'TEXT', '', '', '3000')
    # each grid cell can have up to 5 REs. Loop through the RE columns to get relevant REs with species suitability info
    fields = ['re1', 're2', 're3', 're4', 're5', 're6', 're7', 're8', 're9', 're10', 're11', col]
    with arcpy.da.UpdateCursor(grid, fields) as cursor:
        for row in cursor:
            # update target species column with a list of the unique species with the corresponding NC Act status in that RE
            updateList = []
            for i in range(len(fields)):
                field_name = fields[i]
                field_value = row[i]
                if field_value in reDict:
                    for spec in reDict[field_value]:
                        if spec in subSpecies and spec not in updateList:
                            updateList.append(spec)
            updateList.sort()                
            row[11] = '; '.join(updateList)
            cursor.updateRow(row)

# export species habitat grid
try:
    arcpy.management.CopyFeatures(grid, 'Network_grids/speciesLikelihoodGrid')
except:
    arcpy.management.Delete('Network_grids/speciesLikelihoodGrid')
    arcpy.management.CopyFeatures(grid, 'Network_grids/speciesLikelihoodGrid')

In [4]:
### create bioregional assessment grid

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# create WIP species grid
arcpy.management.CopyFeatures("Network_grids/vegGrid", "Processing/bioAssWIPGrid")
grid = 'bioAssWIPGrid'

nrcols = [field.name for field in arcpy.ListFields('Input/aca_seq_nr_results_v1_1') if not field.required]
rcols = [field.name for field in arcpy.ListFields('Input/aca_seq_r_results_v1_1') if not field.required and field.name not in nrcols]
bpacols = [field.name for field in arcpy.ListFields('Input/seq4_bpa_01') if not field.required]

# create join dictionary
bio = {
    'aca_seq_nr_results_v1_1': {
        'colname': 'aca_nr',
        'fields': nrcols
    },
    'aca_seq_r_results_v1_1': {
        'colname': 'aca_r',
        'fields': rcols
    },
    'seq4_bpa_01': {
        'colname': 'bpa',
        'fields': bpacols
    },
    'seq4_bpa_01_jb': {
        'colname': 'bpa_corridor',
        'fields': ['SIGNIF', 'COR_TYPE']
    }
}

# join attributes to grid using dictionary
i = 0
for dset, attr in bio.items():
    thisLayer = attr['colname']
    arcpy.management.MakeFeatureLayer(f"Input/{dset}", thisLayer)
    if i == 0:
        thisGrid = grid
    else:
        thisGrid = output
    output = f'Processing/{thisLayer}_grid'
    simpleSpatialJoin(thisGrid, thisLayer, output, attr['fields'], 'JOIN_ONE_TO_ONE', 'KEEP_ALL')
    for field in attr['fields']:
        arcpy.management.AlterField(output, field, f'{thisLayer}_{field}', f'{thisLayer}_{field}')
    i += 1

# create binary field for presence of aca stream buffer
arcpy.management.SelectLayerByLocation(output, 'INTERSECT', 'Input/aca_seq_r_results_stream_buffers_v1_1')
arcpy.management.CalculateField(output, 'aca_stream_buffer', '"Yes"', 'PYTHON3')
arcpy.management.SelectLayerByAttribute(output, 'CLEAR_SELECTION')


# copy output grid
try:
    arcpy.management.CopyFeatures(output, 'Network_grids/bioregionalAssessmentGrid')
except:
    arcpy.management.Delete('Network_grids/bioregionalAssessmentGrid')
    arcpy.management.CopyFeatures(output, 'Network_grids/bioregionalAssessmentGrid')

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
NameError: name 'Yes' is not defined


ExecuteError: ERROR 000539: Traceback (most recent call last):
  File "<expression>", line 1, in <module>
NameError: name 'Yes' is not defined

Failed to execute (CalculateField).


In [6]:
# MERGE RRE datasets
## !!!!! SUPERSEDED BY CLEANED RRE DATASET

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")
    
arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# add Caboolture RRE
arcpy.management.MakeFeatureLayer("Input/Refined_Regional_Ecosystem_Caboolture", "rreCaboolture", "current_re <> 'Water'")
# add Narangba RRE
arcpy.management.MakeFeatureLayer("Input/Refined_Regional_Ecosystem_Narangba", "rreNarangba", "current_re <> 'Water'")

# merge two RRE datasets
arcpy.management.Merge("rreCaboolture;rreNarangba", "Processing/rreMerged", "", "ADD_SOURCE_INFO")
# dissolve RRE datasets and obtain minimum bounding geometry
arcpy.analysis.PairwiseDissolve("rreMerged", "Processing/rreDiss")
arcpy.management.MinimumBoundingGeometry("rreDiss", "Processing/minBound", "CONVEX_HULL")

# get statewide RE
arcpy.management.MakeFeatureLayer('Input/Biodiversity_status_of_remnant_regional_ecosystems', "re")

# use the MBG to erase features from statewide RE
arcpy.analysis.PairwiseErase("re", "minBound", "Processing/reErase")
# union the erased re and merged RRE layer
try:
    arcpy.analysis.Union(["re", "rreMerged"], "Input/regionalEcosystemsMerged")
except:
    arcpy.management.Delete("Input/regionalEcosystemsMerged")
    arcpy.analysis.Union(["re", "rreMerged"], "Input/regionalEcosystemsMerged")
    
# clean dataset
keep = ['RE', 'RE1', 'RE2', 'RE3', 'RE4', 'RE5', 'PERCENT', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'RE_LABEL', 'PC_LABEL',
        'LANDZONE', 'BD_STATUS', 'BD_SYMBOL', 'VM_CLASS', 'VM_SYMBOL', 'VM_POLY', 'VERSION', 'L', 'V', 'BVG1M', 'BVG1M_PC', 'DBVG1M', 'DBVG2M', 'DBVG5M',
       'current_re', 'percentage',  'vm_status','MERGE_SRC']

for field in arcpy.ListFields("regionalEcosystemsMerged"):
    if field.name not in keep and not field.required:
        arcpy.management.DeleteField("regionalEcosystemsMerged", field.name)

rename = {
    'current_re': 'refined_re',
    'percentage': 'refined_percent',
    'vm_status': 'refined_vm_status'
}

for old, new in rename.items():
    arcpy.management.AlterField('regionalEcosystemsMerged', old, new, new)

In [29]:
## alternative joins by grid id once grids have already been made

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")
    
arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

grids  = ['bioregionalAssessmentGrid', 'coastalHydroGrid', 'gdeGrid', 'landUseGrid']
for grid in grids:
    gridpath = f'Network_grids/{grid}'
    gridcopy = f'Processing/{grid}_copy'
    arcpy.management.CopyFeatures('Network_grids/vegGrid', gridcopy)
    fields = [field.name for field in arcpy.ListFields(gridpath) if field not in arcpy.ListFields(gridcopy)]
    arcpy.management.JoinField(gridcopy, 'grid_id', gridpath, 'grid_id', fields)

In [57]:
## summarise RE areas supporting MNES communities

arcpy.management.MakeFeatureLayer('Input/regionalEcosystemsMerged', "reMerged")
with arcpy.da.UpdateCursor("re_to_mnes_community", ['Regional_ecosystem', 'mnes_code', 're_area']) as cursor:
    for row in cursor:
        area = 0
        re = row[0]
        query = "RE_combined LIKE '%{}%'".format(re)
        arcpy.management.SelectLayerByAttribute('reMerged', 'NEW_SELECTION', query)
        with arcpy.da.SearchCursor('reMerged', 'Shape_Area') as searchcursor:
            for searchrow in searchcursor:
                area += searchrow[0]
        row[2] = area
        cursor.updateRow(row)